In [151]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import talib
import warnings
warnings.filterwarnings('ignore')

In [152]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

In [153]:
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')
symbols_list = sp500['Symbol'].unique().tolist()

In [154]:
end_date = '2025-06-1'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

In [155]:
df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date)

[*********************100%***********************]  503 of 503 completed


In [156]:
df = df.stack()

In [157]:
df.index.names= ['date','ticker']

In [158]:
df.columns = df.columns.str.lower()

In [159]:
df

Price                   close        high         low        open       volume
date       ticker                                                             
2017-06-05 A        56.733006   57.146291   56.667256   57.108719    1473300.0
           AAPL     35.854847   35.975971   35.745373   35.950349  101326800.0
           ABBV     47.947407   48.195985   47.499960   47.762746    4918100.0
           ABT      40.446312   40.559098   40.238096   40.402934    7810600.0
           ACGL     31.167286   31.420855   31.094383   31.249697    2004900.0
...                       ...         ...         ...         ...          ...
2025-05-30 XYL     126.040001  126.690002  125.199997  126.519997    2624100.0
           YUM     143.940002  144.479996  143.089996  144.039993    3184400.0
           ZBH      91.931602   92.799355   91.203495   92.699615    3505200.0
           ZBRA    289.769989  290.559998  283.920013  289.929993     656100.0
           ZTS     168.630005  169.789993  165.339996  166.919998    5467800.0

[989599 rows x 5 columns]

## 2. Calculate features and technical indicators

In [160]:
df['garmann_klass_vol'] = ((np.log(df['high']) - np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['close']) - np.log(df['open']))**2)
df['rsi'] = df.groupby(level=1)['close'].transform(lambda x: talib.RSI(x.values,timeperiod=20))

df['bb_low']  = df.groupby(level=1)['close'].transform(lambda x: talib.BBANDS(np.log1p(x.values), timeperiod=20)[2])
df['bb_mid']  = df.groupby(level=1)['close'].transform(lambda x: talib.BBANDS(np.log1p(x.values), timeperiod=20)[1])
df['bb_high'] = df.groupby(level=1)['close'].transform(lambda x: talib.BBANDS(np.log1p(x.values), timeperiod=20)[0])

def compute_atr(stock_data):
    atr = talib.ATR(stock_data['high'].values,
                    stock_data['low'].values,
                    stock_data['close'].values,
                    timeperiod=14)
    atr = pd.Series(atr, index=stock_data.index)  # reattach index for alignment
    return (atr - atr.mean()) / atr.std()

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd, signal, hist = talib.MACD(close.values, fastperiod=12, slowperiod=26, signalperiod=9)
    macd = pd.Series(macd, index=close.index)  # maintain alignment
    return (macd - macd.mean()) / macd.std()

df['macd'] = df.groupby(level=1, group_keys=False)['close'].apply(compute_macd)

df['dollar_volume'] = (df['close'] * df['volume'])/1e6

df

Price                   close        high         low        open  \
date       ticker                                                   
2017-06-05 A        56.733006   57.146291   56.667256   57.108719   
           AAPL     35.854847   35.975971   35.745373   35.950349   
           ABBV     47.947407   48.195985   47.499960   47.762746   
           ABT      40.446312   40.559098   40.238096   40.402934   
           ACGL     31.167286   31.420855   31.094383   31.249697   
...                       ...         ...         ...         ...   
2025-05-30 XYL     126.040001  126.690002  125.199997  126.519997   
           YUM     143.940002  144.479996  143.089996  144.039993   
           ZBH      91.931602   92.799355   91.203495   92.699615   
           ZBRA    289.769989  290.559998  283.920013  289.929993   
           ZTS     168.630005  169.789993  165.339996  166.919998   

Price                   volume  garmann_klass_vol        rsi    bb_low  \
date       ticker                                                        
2017-06-05 A         1473300.0           0.000019        NaN       NaN   
           AAPL    101326800.0           0.000018        NaN       NaN   
           ABBV      4918100.0           0.000100        NaN       NaN   
           ABT       7810600.0           0.000031        NaN       NaN   
           ACGL      2004900.0           0.000052        NaN       NaN   
...                        ...                ...        ...       ...   
2025-05-30 XYL       2624100.0           0.000064  57.889330  4.800087   
           YUM       3184400.0           0.000047  44.883265  4.967293   
           ZBH       3505200.0           0.000124  40.059086  4.502665   
           ZBRA       656100.0           0.000267  55.497931  5.520376   
           ZTS       5467800.0           0.000313  59.546814  5.034332   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2017-06-05 A            NaN       NaN       NaN       NaN      83.584737  
           AAPL         NaN       NaN       NaN       NaN    3633.056906  
           ABBV         NaN       NaN       NaN       NaN     235.810141  
           ABT          NaN       NaN       NaN       NaN     315.909964  
           ACGL         NaN       NaN       NaN       NaN      62.487292  
...                     ...       ...       ...       ...            ...  
2025-05-30 XYL     4.834142  4.868197  0.308081  1.028506     330.741566  
           YUM     4.990759  5.014225  0.870221 -0.800426     458.362544  
           ZBH     4.558513  4.614362 -0.158387 -0.923067     322.238653  
           ZBRA    5.648873  5.777371  0.168253  0.964343     190.118090  
           ZTS     5.088457  5.142582  0.629477  1.019474     922.035141  

[989599 rows x 13 columns]

## 3. Aggregate to monthly level and filter 150 most liquid stocks for each month

In [161]:
mean_dollar_vol = df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume')

In [162]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'high','low','open']]

monthly_indicators = df.unstack()[last_cols].resample('M').last().stack('ticker')


In [163]:
data = pd.concat([mean_dollar_vol, monthly_indicators], axis=1).dropna()

Calculate 5-year rolling average of dollar_volume for each stock before filtering

In [164]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

close  garmann_klass_vol        rsi    bb_low  \
date       ticker                                                       
2018-06-30 AAPL     43.787846           0.000252  49.582317  3.782213   
           ABBV     68.034248           0.000205  38.080562  4.215851   
           ABT      53.980724           0.000094  47.018487  3.997916   
           ACN     147.388245           0.000113  58.053311  4.953232   
           ADBE    243.809998           0.000110  50.937227  5.474781   
...                       ...                ...        ...       ...   
2025-05-31 VZ       43.260517           0.000085  53.634953  3.758071   
           WDAY    247.710007           0.000227  48.534163  5.453738   
           WFC      74.779999           0.000080  57.092537  4.286886   
           WMT      98.720001           0.000159  58.713276  4.569311   
           XOM     102.300003           0.000069  42.393078  4.620994   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2018-06-30 AAPL    3.819717  3.857222 -1.242528 -0.279856  
           ABBV    4.278428  4.341006 -0.773602 -1.003932  
           ABT     4.027670  4.057425 -1.337661 -0.351553  
           ACN     4.985015  5.016799 -0.969055  0.031031  
           ADBE    5.521728  5.568676 -1.120731 -0.111269  
...                     ...       ...       ...       ...  
2025-05-31 VZ      3.780491  3.802911  0.429369  0.060829  
           WDAY    5.556007  5.658276  0.711473  0.016955  
           WFC     4.320663  4.354439  1.448850  1.010874  
           WMT     4.588856  4.608402  2.761345  0.948530  
           XOM     4.662747  4.704499  0.718367 -0.987693  

[12516 rows x 8 columns]

## 4. Calculate monthly returns for different time horizons as features

In [165]:
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        df[f'return_{lag}m'] = (df['close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df
    
    
data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

close  garmann_klass_vol        rsi    bb_low  \
date       ticker                                                       
2019-06-30 AAPL     47.530731           0.000071  56.282158  3.781842   
           ABBV     55.922268           0.000223  42.046268  3.983328   
           ABT      75.733635           0.000088  64.665865  4.254858   
           ACN     169.383606           0.000099  58.459437  5.099023   
           ADBE    294.649994           0.000057  58.679794  5.566200   
...                       ...                ...        ...       ...   
2025-05-31 VZ       43.260517           0.000085  53.634953  3.758071   
           WDAY    247.710007           0.000227  48.534163  5.453738   
           WFC      74.779999           0.000080  57.092537  4.286886   
           WMT      98.720001           0.000159  58.713276  4.569311   
           XOM     102.300003           0.000069  42.393078  4.620994   

                     bb_mid   bb_high       atr      macd  return_1m  \
date       ticker                                                      
2019-06-30 AAPL    3.856704  3.931566 -1.103796  0.023163   0.130519   
           ABBV    4.084980  4.186631 -0.416967 -0.931281  -0.052014   
           ABT     4.315521  4.376183 -0.987360  0.935757   0.104689   
           ACN     5.129683  5.160344 -1.072057  0.161058   0.037625   
           ADBE    5.645850  5.725500 -0.887854  0.427627   0.087671   
...                     ...       ...       ...       ...        ...   
2025-05-31 VZ      3.780491  3.802911  0.429369  0.060829  -0.002270   
           WDAY    5.556007  5.658276  0.711473  0.016955   0.011061   
           WFC     4.320663  4.354439  1.448850  1.010874   0.058869   
           WMT     4.588856  4.608402  2.761345  0.948530   0.017570   
           XOM     4.662747  4.704499  0.718367 -0.987693  -0.022605   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
date       ticker                                                          
2019-06-30 AAPL    -0.004965   0.015095   0.039950  -0.013235    0.006858  
           ABBV    -0.042929  -0.029421  -0.032749  -0.024888   -0.016205  
           ABT      0.028136   0.018439   0.026936   0.016743    0.028618  
           ACN      0.005732   0.019101   0.047516   0.011115    0.011659  
           ADBE     0.009291   0.034051   0.045016   0.009775    0.015908  
...                      ...        ...        ...        ...         ...  
2025-05-31 VZ      -0.007679   0.011968   0.004157   0.011156    0.011000  
           WDAY     0.029913  -0.020189  -0.001526  -0.006713    0.013277  
           WFC      0.023408  -0.013502  -0.001336   0.029559    0.020651  
           WMT      0.061706   0.002093   0.012148   0.028495    0.035317  
           XOM     -0.068285  -0.024827  -0.020514  -0.012807   -0.008481  

[10319 rows x 14 columns]

## 5. Download Fama-French Factors and Calculate Rolling-factor betas

In [166]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
              'famafrench', start='2010')[0].drop('RF',axis=1)

In [167]:
factor_data.index = factor_data.index.to_timestamp()
factor_data = factor_data.resample('M').last().div(100)

In [168]:
factor_data

,Mkt-RF,SMB,HML,RMW,CMA
Date,,,,,
2010-01-31,-0.0335,0.0040,0.0033,-0.0108,0.0051
2010-02-28,0.0339,0.0149,0.0318,-0.0029,0.0142
2010-03-31,0.0630,0.0183,0.0219,-0.0061,0.0174
2010-04-30,0.0200,0.0496,0.0296,0.0061,0.0175
2010-05-31,-0.0790,0.0008,-0.0248,0.0130,-0.0024
...,...,...,...,...,...
2025-01-31,0.0280,-0.0122,0.0163,-0.0232,-0.0324
2025-02-28,-0.0243,-0.0491,0.0491,0.0110,0.0306
2025-03-31,-0.0639,-0.0149,0.0290,0.0211,-0.0047


In [169]:
# Step 1: Extract return_1m (MultiIndex: date, ticker)
returns = data['return_1m'] 

# Step 2: Reindex factor_data to match MultiIndex by repeating across tickers
# This matches the index of `returns`
factor_broadcast = factor_data.reindex(returns.index.get_level_values('date')).reset_index()
factor_broadcast.index = returns.index  # Set MultiIndex ['date', 'ticker']

# Step 3: Combine into one DataFrame
factor_data = pd.concat([factor_broadcast, returns], axis=1)

factor_data = factor_data.drop(columns='date')

In [170]:
observations = factor_data.groupby(level=1).size()

valid_stocks = observations[observations >=10]

factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2019-06-30 AAPL    0.0692  0.0037 -0.0064  0.0089 -0.0042   0.130519
           ABBV    0.0692  0.0037 -0.0064  0.0089 -0.0042  -0.052014
           ABT     0.0692  0.0037 -0.0064  0.0089 -0.0042   0.104689
           ACN     0.0692  0.0037 -0.0064  0.0089 -0.0042   0.037625
           ADBE    0.0692  0.0037 -0.0064  0.0089 -0.0042   0.087671
...                   ...     ...     ...     ...     ...        ...
2025-05-31 VZ      0.0606 -0.0072 -0.0288  0.0127  0.0250  -0.002270
           WDAY    0.0606 -0.0072 -0.0288  0.0127  0.0250   0.011061
           WFC     0.0606 -0.0072 -0.0288  0.0127  0.0250   0.058869
           WMT     0.0606 -0.0072 -0.0288  0.0127  0.0250   0.017570
           XOM     0.0606 -0.0072 -0.0288  0.0127  0.0250  -0.022605

[10242 rows x 6 columns]

In [171]:
betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1m'], 
                                     exog=sm.add_constant(x.drop('return_1m', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

betas

Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2019-06-30 AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACN          NaN       NaN       NaN       NaN       NaN
           ADBE         NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2025-05-31 VZ      0.847747 -1.090037  1.035091  0.603548 -0.414251
           WDAY    1.256783 -1.154695  0.450931 -1.696574 -0.051403
           WFC     0.883766 -0.057456  0.437661 -1.667754  0.069469
           WMT     0.522420  0.128991 -0.531404  0.049589 -0.644709
           XOM     0.406687 -0.307038  1.132699  1.104189 -0.251745

[10242 rows x 5 columns]

In [172]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

data = (data.join(betas.groupby('ticker').shift()))

data.loc[:, factors] = data.groupby('ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

data = data.drop('close', axis=1)

data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10112 entries, (Timestamp('2019-06-30 00:00:00'), 'AAPL') to (Timestamp('2025-05-31 00:00:00'), 'XOM')
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   garmann_klass_vol  10112 non-null  float64
 1   rsi                10112 non-null  float64
 2   bb_low             10112 non-null  float64
 3   bb_mid             10112 non-null  float64
 4   bb_high            10112 non-null  float64
 5   atr                10112 non-null  float64
 6   macd               10112 non-null  float64
 7   return_1m          10112 non-null  float64
 8   return_2m          10112 non-null  float64
 9   return_3m          10112 non-null  float64
 10  return_6m          10112 non-null  float64
 11  return_9m          10112 non-null  float64
 12  return_12m         10112 non-null  float64
 13  Mkt-RF             10112 non-null  float64
 14  SMB                10112 non-null  float6